<h1>
<center>Probabilistic Road Traffic Forecasting with Common Machine Learning Classifiers</center>
</h1>

<font size="3">
As mentioned in the report, our goal is to develop a probabilistic forecasting LSTM model for item-store predictions. Currently, in the field of Automation Supply Chain, there is a significant interest in quantitative supply chain approaches that employ probabilistic forecasting. About 95% of these methods generate forecasts using machine learning classifiers with the predict_proba method. In a previous notebbok, we proposed an LSTM model as a superior alternative to classifiers for obtaining probabilities. To validate this, we have created this notebook where we use common machine learning classifiers for probabilistic demand forecasting and measure the results. 
<br>
<br>
Αiming to make as fair a comparison as possible, we perform hyper-parameter optimization οn every algorithm using the validation set. Then knowing the best combination of hyper-parameters we train each model and evaluate the model on test set by measuring various metrics
<br>
<br> 
We obtain the probabilistic forecasts in a distribution format from classifiers, calculate the mean of each output distribution, and compare it with the actual sales recorded. Thus, we treat the mean of the distribution and the actual sales as values and apply regression metrics such as MAE, MSE, MAPE, and R2. This explains why we use regression metrics on classifier outputs.

</font>

## Generals

<font size="3"> 
Packages import and system configurations. 
</font>

In [ ]:
#Data
import os
from os.path import join
import time
import numpy as np
import itertools
from data_handler import FeatureEngineering
import matplotlib.pyplot as plt
from xgboost.sklearn import XGBClassifier
import json


from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier  
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier  # For Random Forest, Gradient Boosting, and AdaBoost
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.neural_network import MLPClassifier  
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

<font size="3"> 
Define necessary paths. 
</font>

In [ ]:
input_path = join("io", "input")
output_path = join("io", "output")
experiments_path = join("io", "experiments")
metrics_path = join(experiments_path, "metrics")
plots_path = join(experiments_path, "plots")

plots_path




## Core Functionality

<font size="3"> 
Load data according to use_validation (if true: return train-validation | else: return train - test) 
</font>

In [ ]:
def load_data(use_validation, feature_cols, training_end_date, validation_end_date, test_end_date):
    
    fe = FeatureEngineering (use_validation, feature_cols, training_end_date, validation_end_date, test_end_date)
    train_set, eval_set, poi_columns = fe.get_datasets()
    x_train, y_train = split_x_y(train_set.head(100), feature_cols+poi_columns) 
    x_eval, y_eval = split_x_y(eval_set.head(100), feature_cols+poi_columns)
    original_classes = sorted(np.unique(y_train))
    desired_classes = list(range(len(original_classes)))
    y_train = [desired_classes[original_classes.index(c)] for c in y_train]
  
    return x_train, y_train, x_eval, y_eval, original_classes





<font size="3"> 
Split: features -  target
</font>

In [ ]:
def split_x_y(df, feature_cols):
    feature_cols = [col for col in feature_cols if col!='node_id']
    y = df['target']
    x = df[feature_cols]
    return x, y

## Hyper-Parameters Tunig

<font size="3">
A core function that apply the evaluation procces using train & validation set:
<ol>
<li>Initialize the given Classification models (KNN, DecisionTree, XGBoost & more) with default hyperparameters.</li>
<li>Define the hyperparameter search spaces (ranges of values to try for each hyperparameter) for each model.</li>
<li>Loop through the three models and their corresponding hyperparameter search spaces.</li>
<li>Generate all possible combinations of hyperparameters for each model.</li>
<li>For each hyperparameter combination, fit the model on the training set, make predictions on the validation set, and calculate the MAE score.</li>
<li>Track the best hyperparameters for each model based on the lowest MAE score on the validation set.</li>
<li>Return the best hyperparameters for each model as a list of dictionaries.</li>

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


In [ ]:
def parameter_tuning_validation(x_train, y_train, x_val, y_val, original_classes):
    # Initialize the models
    tree_cl = DecisionTreeClassifier()
    xgb_cl = XGBClassifier(objective='multi:softprob')
    lm_cl = RidgeClassifier()
    knn_cl = KNeighborsClassifier(weights='distance')
    rf_cl = RandomForestClassifier()
    gb_cl = GradientBoostingClassifier()
    ada_cl = AdaBoostClassifier()
    mpl_cl = MLPClassifier()

    # Define the hyperparameter ranges
    tree_param_grid = {'criterion':['gini', 'entropy'],'max_depth': [12,16,32], 'min_samples_split': [6, 8, 16]}
    xgb_param_grid = {'n_estimators': [6, 12, 24], 'max_depth': [3, 6, 12], 'learning_rate':[0.3,0.5,0.8]}
    lm_param_grid = {'alpha': [0.1, 1.0, 10.0]}
    knn_param_grid = {'n_neighbors': [3, 5, 9, 51, 99], 'algorithm': ['ball_tree', 'kd_tree', 'brute']}
    rf_param_grid = {'max_depth': [8, 12, 24], 'n_estimators': [12, 16, 32], 'min_samples_split': [4, 8, 16]}
    gb_param_grid = {'learning_rate': [0.01, 0.1, 0.5], 'n_estimators': [16, 24, 48], 'max_depth': [3, 6, 12]}
    ada_param_grid = {'learning_rate': [0.001, 0.01, 0.1], 'n_estimators': [30, 40, 60]}
    mpl_param_grid = {'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)], 'activation': ['tanh', 'relu'], 'solver': ['sgd', 'adam'], 'learning_rate': ['constant','adaptive']}

    
    models = [tree_cl, xgb_cl, lm_cl, knn_cl, rf_cl, gb_cl, ada_cl, mpl_cl]
    param_grids = [tree_param_grid, xgb_param_grid, lm_param_grid,
                  knn_param_grid, rf_param_grid, gb_param_grid, ada_param_grid, mpl_param_grid]

    best_params = []
    for i, model in enumerate(models):
        print('\n')
        best_mae = float('inf')
        best_mape = float('inf')
        best_params_i = {}
        # Generate all possible combinations of hyperparameters
        param_combinations = list(itertools.product(*(param_grids[i][param] for param in param_grids[i])))
        for j, params in enumerate(param_combinations):
            # Unpack the tuple of parameter values into individual arguments
            params_dict = dict(zip(param_grids[i], params))
            model.set_params(**params_dict)
            model.fit(x_train, y_train)
            
            if model.__class__.__name__ == 'RidgeClassifier':
                y_val_pred_proba = model.decision_function(x_val)
            else: 
                y_val_pred_proba = model.predict_proba(x_val)

            y_val_pred = np.dot(y_val_pred_proba, original_classes)
            mse = round(mean_squared_error(y_val, y_val_pred), 7)
            mae = round(mean_absolute_error(y_val, y_val_pred),7)
            mape = round(mean_absolute_percentage_error(y_val, y_val_pred), 7)
            r2 = round(r2_score(y_val, y_val_pred),7)
            mse = round(mean_squared_error(y_val, y_val_pred), 7)
            param_str = ', '.join([f'{param}={value}' for param, value in params_dict.items()])
            print(f"Experiment {j+1} with {model.__class__.__name__} using {param_str} has MAE: {mae}, MAPE: {mape}, MSE: {mse}, R2: {r2}")
            if mae < best_mae:
                best_mae = mae
                best_r2 = r2
                best_params_i = dict(zip(param_grids[i], params))
        best_params.append(best_params_i)
    return best_params

## Model Train & Test Evalaution 

<font size="3">
A core function that apply the final train and evaluation procces using train & test set:
<ol>
<li>Initialize the models with the best hyperparameters.</li>
<li>Fit each model on the training data.</li>
<li>Use the fitted models to make predictions on the test set.</li>
<li>Compute the evaluation metrics (MSE, MAE, R2, MAPE) for each model.</li>
<li>Store the results for each model in a dictionary.</li>
<li>Print the evaluation metrics for each model.</li>
<li>Return the dictionary containing the results.</li>

In [ ]:
def evaluate_models(x_train, y_train, x_test, y_test, original_classes, best_params):
    # Initialize the models with the best hyperparameters
    tree_cl = DecisionTreeClassifier(**best_params[0])
    xgb_cl = XGBClassifier(objective='multi:softprob', **best_params[1])
    lm_cl = RidgeClassifier(**best_params[2])
    knn_cl = KNeighborsClassifier(weights='distance', **best_params[3])
    rf_cl = RandomForestClassifier(**best_params[4])
    gb_cl = GradientBoostingClassifier(**best_params[5])
    ada_cl = AdaBoostClassifier(**best_params[6])
    mpl_cl = MLPClassifier(**best_params[7])

    models = [tree_cl, xgb_cl, lm_cl, knn_cl, rf_cl, gb_cl, ada_cl, mpl_cl]
    model_names = ['Decision Tree', 'XGBoost', 'Ridge Classifier',
                  'KNN', 'Random Forest', 'Gradient Boosting', 'AdaBoost', 'MLP Classifier']
    
    results = {}
    for i, model in enumerate(models):
        start_time = time.time()
        model.fit(x_train, y_train)
        
        if model.__class__.__name__ == 'RidgeClassifier':
            y_test_pred_proba = model.decision_function(x_test)
        else: 
            y_test_pred_proba = model.predict_proba(x_test)
                
        y_test_pred = np.dot(y_test_pred_proba, original_classes)
        end_time = time.time()
        execution_time = end_time - start_time
        mse = round(mean_squared_error(y_test, y_test_pred), 7)
        mae = round(mean_absolute_error(y_test, y_test_pred),7)
        r2 = round(r2_score(y_test, y_test_pred),7)
        mape = round(mean_absolute_percentage_error(y_test, y_test_pred), 7)
        results[model_names[i]] = {'MAE': mae,'MAPE': mape, 'MSE': mse, 'R2': r2, 'Execution Time': execution_time}
        print(f"{model_names[i]} model has MAE: {mae}, MAPE: {mape}, MSE: {mse}, R2: {r2}, 'Execution Time': {execution_time}")
    return results

<font size="3"> 
Plot metrics on subplots for camparison purposes
</font>

In [ ]:
def plot_all_metrics_train_val(results,metrics_plot_path):
    model_names = list(results.keys())
    mse = [results[model]['MSE'] for model in model_names]
    mae = [results[model]['MAE'] for model in model_names]
    mape = [results[model]['MAPE'] for model in model_names]
    r2 = [results[model]['R2'] for model in model_names]
    
    x = np.arange(len(model_names))
    width = 0.35
    
    fig, axs = plt.subplots(nrows=2, ncols=2,figsize=(12, 12))
    fig.suptitle('Comparison of Model Performance', fontsize=16)
    # Plot the first metric on the top-left subplot
    axs[0, 0].bar(x - width/2, mse, width, label='MSE')
    axs[0, 0].set_ylabel('MSE')
    axs[0, 0].set_title('MSE')
    axs[0, 0].set_xticks(x)
    axs[0, 0].set_xticklabels(model_names,rotation=90)
    #axs[0, 0].set_ylim(0,1.2)
    axs[0, 0].legend()
    # Plot the second metric on the top-right subplot
    axs[0, 1].bar(x - width/2, mae, width, label='MAE')
    axs[0, 1].set_ylabel('MAE')
    axs[0, 1].set_title('MAE')
    axs[0, 1].set_xticks(x)
    axs[0, 1].set_xticklabels(model_names,rotation=90)
    #axs[0, 1].set_ylim(0,1.2)
    axs[0, 1].legend()
    # Plot the third metric on the bottom-left subplot
    axs[1, 0].bar(x - width/2, mape, width, label='MAPE')
    axs[1, 0].set_ylabel('MAPE')
    axs[1, 0].set_title('MAPE')
    axs[1, 0].set_xticks(x)
    axs[1, 0].set_xticklabels(model_names,rotation=90)
    #axs[1, 0].set_ylim(0,1.2)
    axs[1, 0].legend()
    # Plot the fourth metric on the bottom-right subplot
    axs[1, 1].bar(x - width/2, r2, width, label='R2')
    axs[1, 1].set_ylabel('R2')
    axs[1, 1].set_title('R2')
    axs[1, 1].set_xticks(x)
    axs[1, 1].set_xticklabels(model_names,rotation=90)
    #axs[1, 1].set_ylim(0,1.2)
    axs[1, 1].legend()
    # Adjust the spacing between subplots
    plt.tight_layout()
    # Show the plot
    plt.savefig(metrics_plot_path + '/common_algo_test_metrics.pdf')
    plt.show()

In [ ]:
def save_dict(path, filename, data):
    with open(path + '/' + filename + '.json', 'w') as f:
        json.dump(data, f)

In [ ]:
def load_dict(path, filename):
    with open(path + '/' + filename + '.json', 'r') as f:
        data_loaded = json.load(f)
    return data_loaded

## Pipeline Execution

<font size="3"> 
Hyper-Parameter tuning
</font>

<font size="3"> 
The following function on our machines took a long time to complete (36 hours) so we quote the result below.
</font>

In [ ]:
core_features = ['node_id', 'ETA_curr']
calendar_features = ['cos_hour', 'sin_dayofweek', 'cos_dayofweek', 'sin_month', 'cos_month','sin_dayofmonth', 'cos_dayofmonth', 'sin_year', 'cos_year', 'sin_weekofyear', 'cos_weekofyear', 'sin_quarter_hour', 'cos_quarter_hour']
rolling_avg_features = ['rolling_avg_4h', 'rolling_avg_12h', 'rolling_avg_68h', 'rolling_avg_476h', 'rolling_avg_20240h']
lag_features = ['lag1h', 'lag4h', 'lag476h', 'lag20240h']

feature_cols = core_features + calendar_features + rolling_avg_features + lag_features
training_end_date = '2023-01-01'
validation_end_date = '2024-01-01'
test_end_date = '2025-01-10'


x_train, y_train, x_eval, y_eval, original_classes = load_data(True, feature_cols, training_end_date, validation_end_date, test_end_date)
best_params = parameter_tuning_validation(x_train, y_train, x_eval, y_eval, original_classes)

In [ ]:
# best_params= [
#      {'criterion': 'entropy', 'max_depth': 16, 'min_samples_split': 8},
#      {'n_estimators': 12, 'max_depth': 6, 'learning_rate': 0.5},
#      {'alpha': 1},
#      {'n_neighbors': 9, 'algorithm': 'kd_tree'},
#      {'max_depth': 12, 'n_estimators': 16, 'min_samples_split': 8},
#      {'learning_rate': 0.1, 'n_estimators': 16, 'max_depth': 4},
#      {'learning_rate': 0.01, 'n_estimators': 40},
#      {'hidden_layer_sizes': (50,50,50), 'activation': 'relu', 'solver': 'sgd', 'learning_rate': 'constant'}
# ]

<font size="3"> 
Training and evaluation using testset
</font>

In [ ]:
x_train, y_train, x_test, y_test, original_classes = load_data(False, feature_cols, training_end_date, validation_end_date, test_end_date)
results =  evaluate_models(x_train, y_train, x_test, y_test, original_classes, best_params)
save_dict(metrics_path, 'ml_algo_test_metrics', results)

<font size="3"> 
Plot results for comparison purposes 
</font>

In [ ]:
metrics_path=load_dict(metrics_path, 'ml_algo_test_metrics')

In [ ]:
plot_all_metrics_train_val(a, plots_path)